# Работа с открытым API HeadHunter

**Цель:**
отладка параметров запросов к HH, формат загружаемых данных

## 1. Импорт библиотек и установка констант

In [116]:

import requests
import pandas as pd
import time
import datetime as dt

from pathlib import Path
from bs4 import BeautifulSoup
from tqdm import tqdm

In [166]:
# Создадим хранилище для сырых данных
RAW_DIR = Path('../data/raw_data')
RAW_DIR.mkdir(parents=True, exist_ok=True)

MASTER_PATH = Path("../data/raw_data/vacancies_master.csv")

MASTER_COLUMNS = [
    "id", "name", "area", "experience",
    "key_skills", "description"
]

# Эндпоинт поиска вакансий через API
BASE_URL = 'https://api.hh.ru/vacancies'


# Параметры поиска по умолчанию
DEFAULT_PARAMS = {"text":"name:(data analyst) OR name:(data scientist)",
                  "area":"113", # Россия
                  "per_page": 100, # вакансий на страницу
                  "page": 0, # номер страницы
                  "only_with_salary": False, # не только с указанной зарплатой
                  "search_field": "name", # поле для поиска текста
                  "date_from": None, "date_to": None # фильтры по дате
}




Создадим запрос с базовыми параметрами для проверки:

In [17]:
requests.get(BASE_URL, params=DEFAULT_PARAMS).json()

{'items': [{'id': '127202494',
   'premium': False,
   'name': 'ML-аналитик / Data Scientist',
   'department': None,
   'has_test': False,
   'response_letter_required': False,
   'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
   'salary': None,
   'salary_range': None,
   'type': {'id': 'open', 'name': 'Открытая'},
   'address': None,
   'response_url': None,
   'sort_point_distance': None,
   'published_at': '2025-11-01T09:09:01+0300',
   'created_at': '2025-11-01T09:09:01+0300',
   'archived': False,
   'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=127202494',
   'show_logo_in_search': None,
   'show_contacts': False,
   'insider_interview': None,
   'url': 'https://api.hh.ru/vacancies/127202494?host=hh.ru',
   'alternate_url': 'https://hh.ru/vacancy/127202494',
   'relations': [],
   'employer': {'id': '55551',
    'name': 'ЮНИСТРИМ БАНК',
    'url': 'https://api.hh.ru/employers/55551',
    'alternate_url': 'https://hh.ru/em

В списке вакансий есть только "верхнеуровневые данные". Чтобы получить "скилы", нужно "дергать" каждую вакансию по отдельности:

In [18]:
requests.get('https://api.hh.ru/vacancies/126905966').json()

{'id': '126905966',
 'premium': False,
 'billing_type': {'id': 'standard_plus', 'name': 'Стандарт плюс'},
 'relations': [],
 'name': 'Data Scientist (junior)',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': None,
 'salary_range': None,
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Москва',
  'street': 'проспект Лихачёва',
  'building': '15',
  'lat': 55.699872,
  'lng': 37.644261,
  'description': None,
  'raw': 'Москва, проспект Лихачёва, 15',
  'metro': {'station_name': 'Автозаводская',
   'line_name': 'Замоскворецкая',
   'station_id': '2.2',
   'line_id': '2',
   'lat': 55.706634,
   'lng': 37.657008},
  'metro_stations': [{'station_name': 'Автозаводская',
    'line_name': 'Замоскворецкая',
    'station_id': '2.2',
    'line_id': '2',
    'lat': 55.706634,
    'lng': 37.657008},
   {'station_name': 'ЗИЛ',
    'line_name': 'Троицкая',
    'station_id': '137.96

**Итого, подход к сбору данных:**
* Собираем данные о вакансиях, соответствующих заданному условию (по дате, региону, наименованию)
* Для каждой полученной вакансии "забираем" данные о ключевых навыках и описание вакансии (описание - на случай, если навыки не указаны, тогда их можно будет извлечь с помощью LLM).

# Теперь проработаем сохранение необходимых данныx в pandas DataFrame

In [19]:
# сохраним словарь со списком вакансий:
vac_list = requests.get(BASE_URL, params=DEFAULT_PARAMS).json()['items']

Посмотрим несколько вакансий

In [20]:
vac_list[2]

{'id': '127192646',
 'premium': False,
 'name': 'Младший аналитик данных/Junior Data Analyst',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': None,
 'salary_range': None,
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Москва',
  'street': 'улица Двинцев',
  'building': '12к1Б',
  'lat': 55.796764,
  'lng': 37.599013,
  'description': None,
  'raw': 'Москва, улица Двинцев, 12к1Б',
  'metro': {'station_name': 'Марьина Роща',
   'line_name': 'Люблинско-Дмитровская',
   'station_id': '10.185',
   'line_id': '10',
   'lat': 55.793723,
   'lng': 37.61618},
  'metro_stations': [{'station_name': 'Марьина Роща',
    'line_name': 'Люблинско-Дмитровская',
    'station_id': '10.185',
    'line_id': '10',
    'lat': 55.793723,
    'lng': 37.61618},
   {'station_name': 'Савёловская',
    'line_name': 'Серпуховско-Тимирязевская',
    'station_id': '9.128',
    'line

In [21]:
vac_list[5]

{'id': '126209623',
 'premium': False,
 'name': 'Стажер, Data Analyst / Data Scientist',
 'department': {'id': '201-201-itserv', 'name': 'Kept, ИТ-сервисы'},
 'has_test': True,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': None,
 'salary_range': None,
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Москва',
  'street': 'Ленинградский проспект',
  'building': '34А',
  'lat': 55.78531580767879,
  'lng': 37.567994114085565,
  'description': None,
  'raw': 'Москва, Ленинградский проспект, 34А',
  'metro': {'station_name': 'Динамо',
   'line_name': 'Замоскворецкая',
   'station_id': '2.34',
   'line_id': '2',
   'lat': 55.789704,
   'lng': 37.558212},
  'metro_stations': [{'station_name': 'Динамо',
    'line_name': 'Замоскворецкая',
    'station_id': '2.34',
    'line_id': '2',
    'lat': 55.789704,
    'lng': 37.558212},
   {'station_name': 'Петровский парк',
    'line_name': 'Большая кольцевая 

На первый взгляд интерес представляют поля:
1. `id`
2. `name`
3. `area['name']`
4. `experience['name']`

Детальные данные по скилам и требованиям будем получать уже непосредственно из описаний вакансий

In [36]:
vac_base = pd.DataFrame(columns=['id','name','area','experience'])

In [37]:
for vac in vac_list:
    vac_base.loc[len(vac_base)] = [
        vac.get('id'),
        vac.get('name'),
        (vac.get('area') or {}).get('name'),
        (vac.get('experience') or {}).get('name'),
    ]


In [40]:
vac_base.sample(5)

,id,name,area,experience
44,127138420,Data аналитик (AdTech),Москва,От 3 до 6 лет
45,127181192,Data Scientist,Москва,От 3 до 6 лет
70,126776801,Data Scientist,Москва,От 1 года до 3 лет
54,126920458,Аналитик данных / Data Analyst,Москва,От 1 года до 3 лет
80,126231402,"Data Scientist, NLP, Ozon Банк",Москва,От 1 года до 3 лет


Собрали список. Следующий шаг - по id вакансии добавить в таблицу навыки и описание (поля `key_skills` и `description`)

In [68]:
i=0
for id in vac_base['id']:
    resp = BASE_URL+'/'+str(id) # Создаем ссылку на вакансию
    vac = requests.get(resp).json()
    time.sleep(0.5) # Задержка обращения по API
    print(i,end='. ')
    for el in vac['key_skills']:
        print(el['name'],end=', ')
    soup = BeautifulSoup(vac['description'], 'html.parser')
    print(soup.get_text())
    print()
    i+=1
    if i == 5:
        break

0. Обязанности:  Моделирование клиентского поведения, прогнозирование оттока; Работа в Yandex DataSphere, создание и тренировка ML-моделей; Подготовка данных для моделей (ETL-процессы, очистка, фичи); Построение системы «цифрового двойника клиента».  Требования:  Уверенное владение Python, pandas, scikit-learn, MLflow; Опыт работы в DataSphere или аналогичных средах; Знание SQL и базовых принципов ETL; Понимание клиентской аналитики, NPS, retention.  Условия:  Интересные задачи и проекты; Оформление по ТК РФ; Официальная заработная плата (уровень оплаты труда, обсуждается с успешным кандидатом на собеседовании); График работы 5/2 пн-чт с 09.30 до 18.15, пятница с 09.30 до 17.00. 

1. Python, SQL, Базы данных, Анализ данных, Ищем специалиста, готового влиться в команду профессионалов в один из крупнейших банков страны. Сегодня мы в поиске Аналитика данных. Основные обязанности:  Работа в функциональной команде, нацеленной на развитие продаж Работа с базами данных с целью оцифровки и раз

In [81]:
# Разобрались с "выдергиванием" навыков и описания вакансии, теперь соберем DataFrame
vac_details = [] # список для хранения
# i = 0
for id in vac_base['id']:
    # i+=1
    resp = BASE_URL+'/'+str(id)
    vac = requests.get(resp).json()
    time.sleep(1) # при задержке в 0.5 секунды сессия обрывалась, увеличил до одной секунды, на 100 вакансиях отработала

    # достаем навыки
    skills = ', '.join([el['name'] for el in vac.get('key_skills',[])])

    # чистим текст
    description = BeautifulSoup(vac.get('description', ''), 'html.parser').get_text().strip()

    # добавляем данные в список словарей
    vac_details.append({
        'id':id,
        'key_skills':skills,
        'description':description
    })
    # if i == 5:
    #     break
vac_df = pd.DataFrame(vac_details)
vac_df.sample(5)

,id,key_skills,description
91,127194992,,Чем предстоит заниматься: Анализ пользовател...
61,126786396,"SQL, Power BI, Бизнес-анализ",Finstar Financial Group – крупный международны...
69,125390186,"Python, PyTorch, TensorFlow, Self-Supervised L...",Computer Vision. Разработка и оптимизация алго...
54,126920458,"Аналитическое мышление, Анализ данных, MS Exce...","АНО ""Диалог Регионы"" – федеральная компания, у..."
12,126826034,"SQL, MS SQL, Power BI, Визуализация данных, Фо...",Компания AMCOR – отечественная инжиниринговая ...


In [87]:
# объединим полученные таблицы по id
vac_full = pd.merge(vac_base, vac_df, on='id', how='left')
vac_full.sample(5)

,id,name,area,experience,key_skills,description
62,124119703,Data scientist Geospatial Analyst,Москва,От 1 года до 3 лет,,Обязанности:• Создание и непрерывное улучшение...
95,120549449,Data analyst (Сервисы роботизации),Москва,От 1 года до 3 лет,,Объединённая компания Wildberries и Russ — это...
14,126315420,Data Analyst,Москва,От 3 до 6 лет,"SQL, Python, Анализ данных","Robusta — международная IT компания, которая р..."
54,126920458,Аналитик данных / Data Analyst,Москва,От 1 года до 3 лет,"Аналитическое мышление, Анализ данных, MS Exce...","АНО ""Диалог Регионы"" – федеральная компания, у..."
89,127184837,Data Scientist,Москва,От 3 до 6 лет,,"Мы создаём продукт, который меняет то, как FMC..."


Получилось:
* Вытащить список вакансий по заданным параметрам
* Для каждой вакансии получить поля "ключевые навыки" и "описание вакансии"

Следующий шаг - написание функции для системного сбора данных о вакансиях

In [109]:
def vacancyParser(url=BASE_URL, params=None, pause=1.5, max_pages=None):
    base_params = (params or DEFAULT_PARAMS).copy()

    # первичный запрос: узнаём число страниц
    r = requests.get(url, params=base_params)
    r.raise_for_status()
    data = r.json()
    pages = data.get('pages', 0)

    if max_pages is not None:
        pages = min(pages, max_pages)

    # собираем базовую таблицу вакансий
    rows = []
    for page in tqdm(range(pages), total=pages, desc="Страницы"):
        page_params = base_params.copy()
        page_params['page'] = page

        resp = requests.get(url, params=page_params)
        resp.raise_for_status()
        items = resp.json().get('items', [])

        for vac in items:
            rows.append({
                'id': vac.get('id'),
                'name': vac.get('name'),
                'area': (vac.get('area') or {}).get('name'),
                'experience': (vac.get('experience') or {}).get('name'),
            })

        time.sleep(pause)

    vacancyBase = pd.DataFrame(rows, columns=['id','name','area','experience'])

    # собираем детали по каждой вакансии
    details = []
    for vid in tqdm(vacancyBase['id'], desc="Детали", total=len(vacancyBase)):
        resp = requests.get(f"{url}/{vid}")
        if resp.status_code != 200:
            continue
        vacancy = resp.json()
        time.sleep(pause)

        skills_list = ', '.join([el.get('name') for el in (vacancy.get('key_skills') or [])])

        # Чистим HTML в описании
        description = BeautifulSoup(
            vacancy.get('description', '') or '',
            'html.parser'
        ).get_text().strip()

        details.append({
            'id': vid,
            'key_skills': skills_list,
            'description': description
        })

    details_df = pd.DataFrame(details, columns=['id','key_skills','description'])

    # объединяем
    if not details_df.empty:
        vacancyBase = vacancyBase.merge(details_df, on='id', how='left')
    else:
        vacancyBase['key_skills'] = [[] for _ in range(len(vacancyBase))]
        vacancyBase['description'] = ""

    return vacancyBase



In [111]:
vb = vacancyParser()
vb.sample(5)

Детали: 100%|██████████| 398/398 [12:07<00:00,  1.83s/it]


,id,name,area,experience,key_skills,description
325,126795918,Data Scientist (Senior),Москва,От 3 до 6 лет,,«М.ТЕХ» - АККРЕДИТОВАННАЯ ИТ-КОМПАНИЯ В ГРУППЕ...
174,124803044,Data Scientist,Москва,От 1 года до 3 лет,,Группа компаний Протектор – ведущий поставщик ...
225,126119482,Data Scientist Middle/Middle+,Москва,От 1 года до 3 лет,,"Обязанности:-Разрабатывать, внедрять и оптимиз..."
168,126314693,Data Scientist (Блок Финансы),Москва,От 1 года до 3 лет,,Мы развиваем решения в области uplift-моделиро...
302,125969338,Data Scientist,Москва,От 1 года до 3 лет,,Вам предстоит: Построение и обучение моделей ...


Собрали данные по 398 вакансиям направления `data analyst`, `data scientist`.
Далее:
* Сохраним датасет
* Впоследствии будем получать только данные по новым вакансиям, которых нет в базе и пополнять нашу базу вакансий

In [121]:
def saveVacanciesToCsv(df, dirPath = RAW_DIR, prefix = 'Vacancies_on'):
    outDir = Path(dirPath)
    outDir.mkdir(parents=True, exist_ok=True)

    ts = dt.datetime.today().strftime('%Y%m%d')
    fname = outDir / f"{prefix}_{ts}.csv"

    df.to_csv(fname, index=False, encoding='utf-8-sig')

    print(f'Данные сохранены в CSV: {fname}')

In [122]:
saveVacanciesToCsv(df=vb) #Сохранили полученную базу вакансий в csv

Данные сохранены в CSV: ..\data\raw_data\Vacancies_on_20251104.csv


Сделаем следующие шаги:
* Создадим отдельный файл с мастер-данными, содержащий все собранные данные
* Создадим функцию новых данных, работающую след. образом:
    * Берем список id из мастер-файла
    * Получаем запросом список вакансий
    * Только по вакансиям, отсутствующим в мастер файле парсим навыки и описание
    * Создаем новый CSV файл
    * Добавляем данные в мастер-файл.

In [127]:
# Сохраним собранные данные в мастер-файл

vb.to_csv(RAW_DIR / 'vacancies_master.csv', index=False, encoding='utf-8-sig')

In [170]:
# Проверяем наличие/создаем мастер-файл
def ensureMaster(master_path: Path = MASTER_PATH):
    master_path.parent.mkdir(parents=True, exist_ok=True)
    if not master_path.exists():
        pd.DataFrame(columns=MASTER_COLUMNS).to_csv(
            master_path, index=False, encoding="utf-8-sig"
        )
        print(f"Создан пустой мастер-файл: {master_path}")
    else:
        print(f"Мастер-файл найден: {master_path}")

# Получаем id имеющихся вакансий
def getMasterIds(master_path: Path = MASTER_PATH) -> set:
    ensureMaster(master_path)
    df = pd.read_csv(master_path, dtype=str)
    return set(df["id"].dropna().astype(str).tolist())

In [171]:
# Получаем список вакансий
def fetchVacansiesList(params = DEFAULT_PARAMS, max_pages = None, pause = 1.5):
    r = requests.get(BASE_URL, params=params)
    r.raise_for_status()
    data = r.json()
    pages_total = data.get('pages', 0)

    if max_pages is not None:
        pages_total = min(pages_total, max_pages)

    rows = []
    for page in tqdm(range(pages_total), desc="Поиск"):
        page_params = {**params, "page": page}
        resp = requests.get(BASE_URL, params=page_params)
        resp.raise_for_status()
        items = resp.json().get("items", [])
        for it in items:
            rows.append({
                "id": it.get("id"),
                "name": it.get("name"),
                "area": (it.get("area") or {}).get("name"),
                "experience": (it.get("experience") or {}).get("name"),
            })
        time.sleep(pause)

    return pd.DataFrame(rows, columns=["id","name","area","experience"])

# Оставляем те, которых нет в мастере
def filterNewIds(df_basic: pd.DataFrame, master_ids: set) -> pd.DataFrame:

    df_basic = df_basic.astype({"id": str})
    mask_new = ~df_basic["id"].isin(master_ids)
    return df_basic.loc[mask_new].drop_duplicates(subset=["id"])

In [172]:
master_ids = getMasterIds()

params = DEFAULT_PARAMS.copy()
params["page"] = 0

df_basic = fetchVacansiesList(params=params, max_pages=5)
df_new_basic = filterNewIds(df_basic, master_ids)

print(f"Найдено в поиске: {len(df_basic)}")
print(f"Новых (ещё не в мастере): {len(df_new_basic)}")

Мастер-файл найден: ..\data\raw_data\vacancies_master.csv


Поиск: 100%|██████████| 4/4 [00:17<00:00,  4.39s/it]

Найдено в поиске: 397
Новых (ещё не в мастере): 0


In [176]:
# Функция загрузки данных по новым вакансиям

def fetchVacanciesDetails(ids, pause = 1.5, base_url = BASE_URL):
    rows = []
    for id in ids:
        r = requests.get(f"{base_url}/{id}")
        if r.status_code != 200:
            time.sleep(pause)
            continue
        v = r.json()

        key_skills = ", ".join([k.get("name") for k in (v.get("key_skills") or [])])

        desc_html = (v.get("description") or "")
        description = BeautifulSoup(desc_html, "html.parser").get_text(" ").strip()

        rows.append({
            "id": str(v.get("id")),
            "key_skills": key_skills,
            "description": description
        })
        time.sleep(pause)

        return pd.DataFrame(rows, columns=["id","key_skills","description"])


In [177]:
# функция сборки датасета из новых вакансий

def build_new_dataset(df_new_basic: pd.DataFrame) -> pd.DataFrame:
    # берём список новых id
    new_ids = df_new_basic["id"].astype(str).tolist()
    # тянем детали
    df_details = fetchVacanciesDetails(new_ids, pause=0.3)
    # объединяем
    if df_details.empty:
        # если деталей не получили (редко), вернём хотя бы basic с пустыми полями
        df_new_full = df_new_basic.copy()
        df_new_full["key_skills"] = ""
        df_new_full["description"] = ""
        return df_new_full

    df_new_full = (
        df_new_basic.astype({"id": str})
        .merge(df_details, on="id", how="left")
    )
    return df_new_full[["id","name","area","experience","key_skills","description"]]

In [182]:
# Добавление данных в мастер-файл

def append_to_master(df_new_full: pd.DataFrame, master_path=MASTER_PATH):
    ensureMaster(master_path)
    df_master = pd.read_csv(master_path, dtype=str)

    before = len(df_master)
    # конкат и удаление дубликатов по id
    df_updated = (
        pd.concat([df_master, df_new_full.astype(str)], ignore_index=True)
          .drop_duplicates(subset=["id"])
    )
    df_updated.to_csv(master_path, index=False, encoding="utf-8-sig")
    added = len(df_updated) - before
    print(f"Мастер обновлён: +{added} строк, всего {len(df_updated)} → {master_path}")
    return added

In [181]:
master_ids = getMasterIds()
df_basic = fetchVacansiesList(params, max_pages=20)
df_new_basic = filterNewIds(df_basic, master_ids)

if len(df_new_basic) > 0:
    df_new_full = build_new_dataset(df_new_basic)
    new_csv_path = saveVacanciesToCsv(df_new_full)         # (3.3)
    _added = append_to_master(df_new_full)           # (3.4)
else:
    print("Новых вакансий нет — шаг 3 пропущен.")

Мастер-файл найден: ..\data\raw_data\vacancies_master.csv


Поиск: 100%|██████████| 4/4 [00:16<00:00,  4.13s/it]

Новых вакансий нет — шаг 3 пропущен.


Парсер для необходимых данных готов. Перенесем код в отдельный скрипт